In [1]:
import os
os.chdir('/home/dataproc/production')
import inverted_index_gcp
import se
from datetime import datetime
from se import search_engine
import importlib
importlib.reload(inverted_index_gcp)
importlib.reload(se)

def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load engine
cur_se = search_engine()
cur_se.init_engine(load_only_title=False)

loading title indecies
loading rest of the indecies
loading page views
loading page rank


In [3]:
# load test queries
import json
with open('new_train.json') as json_file:
# with open('queries_train_last_year.json') as json_file:
    official_queries = json.load(json_file)

In [4]:
# current params
params = {'max_docs_from_binary_title': 4433,
            'max_docs_from_binary_body': 1349,
            'bm25_body_weight': 5.175842946028495,
            'bm25_title_weight': 2.297919942629382,
            'bm25_body_bi_weight': 0.9433857458903415,
            'bm25_title_bi_weight': 5.43406860036612,
            'body_cosine_score': 4.925136475255385,
            'title_cosine_score': 0.29963143927827507,
            'pr_weight': 0.34839354014586377,
            'pv_weight': 4.740913798917137}

In [5]:
# run one query
from datetime import datetime
test_query = 'python'
print('query:',test_query)
start = datetime.now()
b = cur_se.full_search_(test_query,params)
q2 = cur_se.search_body_Q2(test_query)
q3 = cur_se.search_title_Q3(test_query)
q4 = cur_se.search_anchor_Q4(test_query)
q5 = cur_se.page_rank_Q5([901126,6111238,48407627,30796675])
q6 = cur_se.page_view_Q6([901126,6111238,48407627,30796675])

end = datetime.now()
total_time = (end - start).total_seconds()

# our_results = list(zip(*b))[0][:100]
# offical_results = official_queries[test_query]
# print('average_precision',average_precision(our_results,offical_results))
print('total time',total_time)

query: python
title_candidates: 158
body_candidates: 8428
body_candidates_bi_gram: 0
title_candidates_bi_gram: 0
after binary search title_candidates: 158
after binary search body_candidates: 1349
anchor_candidates: 0
cos_candidates_to_add: 0
total search time 0.164299 seconds
total time 0.20636


In [8]:
# run profile
# %prun -s cumtime cur_se.full_search_('best marvel movie',params)

In [11]:
import os
os.chdir('/home/dataproc/production')
import inverted_index_gcp
import se
from datetime import datetime
from se import search_engine
import importlib
importlib.reload(inverted_index_gcp)
importlib.reload(se)

def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

def run_se_test():
    # load test queries
    import json
    with open('new_train.json') as json_file:
        official_queries = json.load(json_file)
    # run list of queries with performance monitor
    params = {'max_docs_from_binary_title': 4433,
                'max_docs_from_binary_body': 1349,
                'bm25_body_weight': 5.175842946028495,
                'bm25_title_weight': 2.297919942629382,
                'bm25_body_bi_weight': 0.9433857458903415,
                'bm25_title_bi_weight': 5.43406860036612,
                'body_cosine_score': 4.925136475255385,
                'title_cosine_score': 0.29963143927827507,
                'pr_weight': 0.34839354014586377,
                'pv_weight': 4.740913798917137}

    import json
    # with open('new_train.json') as json_file:
    with open('queries_train_last_year.json') as json_file:
        official_queries = json.load(json_file)

    avg_pre = 0
    counter = 0
    avg_time = 0
    max_time = 0

    for key, value in official_queries.items():
        test_query = key
        print('query:',test_query)
        start = datetime.now()
        b = cur_se.full_search_(test_query,params)
        end = datetime.now()
        total_time = (end - start).total_seconds()
        avg_time +=total_time
        max_time = max(max_time,total_time)

        our_results = list(zip(*b))[0][:99]
        offical_results = official_queries[test_query]
        avg_pre +=average_precision(our_results,offical_results)
        counter +=1
        print('average_precision',average_precision(our_results,offical_results))
        print('*********')
    print('final score:{}'.format(avg_pre/counter))
    print('avg_time score:{}'.format(avg_time/counter))
    print('max_time score:{}'.format(max_time))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
run_se_test()

query: python
title_candidates: 158
body_candidates: 8428
body_candidates_bi_gram: 0
title_candidates_bi_gram: 0
after binary search title_candidates: 158
after binary search body_candidates: 1349
anchor_candidates: 0
cos_candidates_to_add: 0
total search time 0.039422 seconds
average_precision 0.808
*********
query: data science
title_candidates: 9323
body_candidates: 519863
body_candidates_bi_gram: 1543
title_candidates_bi_gram: 13
after binary search title_candidates: 4433
after binary search body_candidates: 1543
anchor_candidates: 0
cos_candidates_to_add: 0
total search time 0.620384 seconds
average_precision 0.628
*********
query: migraine
title_candidates: 25
body_candidates: 1589
body_candidates_bi_gram: 0
title_candidates_bi_gram: 0
after binary search title_candidates: 25
after binary search body_candidates: 1349
anchor_candidates: 0
cos_candidates_to_add: 0
total search time 0.019393 seconds
average_precision 0.838
*********
query: chocolate
title_candidates: 483
body_candid

In [ ]:
# run hyperparameters
import optuna

def objective(trial):
    
    param = {'max_docs_from_binary_title' :trial.suggest_int("max_docs_from_binary_title",200 , 5000),
              'max_docs_from_binary_body' :trial.suggest_int("max_docs_from_binary_body",200 , 5000),
              'bm25_body_weight':trial.suggest_float("bm25_body_weight", 0, 10),
              'bm25_title_weight':trial.suggest_float("bm25_title_weight", 0, 10),
              'bm25_body_bi_weight':trial.suggest_float("bm25_body_bi_weight", 0, 10),
              'bm25_title_bi_weight':trial.suggest_float("bm25_title_bi_weight", 0, 10),
              'body_cosine_score': trial.suggest_float("body_cosine_score", 0, 10),
              'title_cosine_score': trial.suggest_float("title_cosine_score", 0, 10),
              'pr_weight':trial.suggest_float("pr_weight", 0, 10),
              'pv_weight':trial.suggest_float("pv_weight", 0, 10)
         }

    avg_pre = 0
    counter = 0
    for key, value in official_queries.items():
        test_query = key
        b = cur_se.full_search_(test_query,param)
        our_results = list(zip(*b))[0][:40]
        offical_results = official_queries[test_query]
        avg_pre +=average_precision(our_results,offical_results)
        counter +=1
#         print('average_precision',average_precision(our_results,offical_results))
#         print('*********')
    final_score = avg_pre/counter
    print('final score:{}'.format(final_score))
    return final_score


# run hyperparameters
from optuna.samplers import TPESampler

study = optuna.create_study(sampler=TPESampler(), direction="maximize")
study.optimize(objective, n_trials=150, timeout=6000) # Run for 100 minutes
print("Number of completed trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("\tBest Score: {}".format(trial.value))
print("\tBest Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-01-15 06:21:41,537] A new study created in memory with name: no-name-1d46a522-0ed1-4f33-81f8-dbb61cf0bc12


title_candidates: 9759
body_candidates: 813413
body_candidates_bi_gram: 112
title_candidates_bi_gram: 0
after binary search title_candidates: 4014
after binary search body_candidates: 721
anchor_candidates: 0
cos_candidates_to_add: 0
title_candidates: 42617
body_candidates: 570026
body_candidates_bi_gram: 1624
title_candidates_bi_gram: 10
after binary search title_candidates: 4014
after binary search body_candidates: 1624
anchor_candidates: 0
cos_candidates_to_add: 0
title_candidates: 2387
body_candidates: 471533
body_candidates_bi_gram: 1400
title_candidates_bi_gram: 33
after binary search title_candidates: 2387
after binary search body_candidates: 1400
anchor_candidates: 0
cos_candidates_to_add: 0
title_candidates: 7
body_candidates: 2393
body_candidates_bi_gram: 0
title_candidates_bi_gram: 0
after binary search title_candidates: 7
after binary search body_candidates: 721
anchor_candidates: 0
cos_candidates_to_add: 0
title_candidates: 655
body_candidates: 35726
body_candidates_bi_gra

[I 2023-01-15 06:22:08,335] Trial 0 finished with value: 0.48468965517241375 and parameters: {'max_docs_from_binary_title': 4014, 'max_docs_from_binary_body': 721, 'bm25_body_weight': 4.294903625921274, 'bm25_title_weight': 9.142192973028049, 'bm25_body_bi_weight': 0.9673210914827302, 'bm25_title_bi_weight': 0.03991428305181177, 'body_cosine_score': 5.166845485183547, 'title_cosine_score': 0.9529646452696716, 'pr_weight': 4.013355716243121, 'pv_weight': 2.215794644666845}. Best is trial 0 with value: 0.48468965517241375.


after binary search title_candidates: 1649
after binary search body_candidates: 721
anchor_candidates: 0
cos_candidates_to_add: 0
final score:0.48468965517241375
title_candidates: 9759
body_candidates: 813413
body_candidates_bi_gram: 112
title_candidates_bi_gram: 0
after binary search title_candidates: 3854
after binary search body_candidates: 4787
anchor_candidates: 0
cos_candidates_to_add: 0
title_candidates: 42617
body_candidates: 570026
body_candidates_bi_gram: 1624
title_candidates_bi_gram: 10
after binary search title_candidates: 3854
after binary search body_candidates: 1624
anchor_candidates: 0
cos_candidates_to_add: 0
title_candidates: 2387
body_candidates: 471533
body_candidates_bi_gram: 1400
title_candidates_bi_gram: 33
after binary search title_candidates: 2387
after binary search body_candidates: 1400
anchor_candidates: 0
cos_candidates_to_add: 0
title_candidates: 7
body_candidates: 2393
body_candidates_bi_gram: 0
title_candidates_bi_gram: 0
after binary search title_candi

[I 2023-01-15 06:22:39,379] Trial 1 finished with value: 0.5478620689655174 and parameters: {'max_docs_from_binary_title': 3854, 'max_docs_from_binary_body': 4787, 'bm25_body_weight': 1.7833224189556485, 'bm25_title_weight': 1.4458203844163853, 'bm25_body_bi_weight': 1.2423672391133067, 'bm25_title_bi_weight': 3.594848886515835, 'body_cosine_score': 8.939565324338737, 'title_cosine_score': 1.4497113645781035, 'pr_weight': 1.431715353981784, 'pv_weight': 4.206838244988026}. Best is trial 1 with value: 0.5478620689655174.


...

final score:0.5644137931034482
title_candidates: 9759
body_candidates: 813413
body_candidates_bi_gram: 112
title_candidates_bi_gram: 0
after binary search title_candidates: 1159
after binary search body_candidates: 4669
anchor_candidates: 0
cos_candidates_to_add: 0
title_candidates: 42617
body_candidates: 570026
body_candidates_bi_gram: 1624
title_candidates_bi_gram: 10
after binary search title_candidates: 1159
after binary search body_candidates: 1624
anchor_candidates: 0
cos_candidates_to_add: 0
title_candidates: 2387
body_candidates: 471533
body_candidates_bi_gram: 1400
title_candidates_bi_gram: 33
after binary search title_candidates: 1159
after binary search body_candidates: 1400
anchor_candidates: 0
cos_candidates_to_add: 0
title_candidates: 7
body_candidates: 2393
body_candidates_bi_gram: 0
title_candidates_bi_gram: 0
after binary search title_candidates: 7
after binary search body_candidates: 2393
anchor_candidates: 0
cos_candidates_to_add: 0
title_candidates: 655
body_candida

[I 2023-01-15 07:29:32,446] Trial 148 finished with value: 0.5665862068965517 and parameters: {'max_docs_from_binary_title': 1159, 'max_docs_from_binary_body': 4669, 'bm25_body_weight': 7.097575259712054, 'bm25_title_weight': 8.33782582711889, 'bm25_body_bi_weight': 3.1658206411741023, 'bm25_title_bi_weight': 7.508164219544747, 'body_cosine_score': 9.9603114241189, 'title_cosine_score': 4.38586167211175, 'pr_weight': 1.0693864875777566, 'pv_weight': 5.0487977977458165}. Best is trial 146 with value: 0.5772068965517242.


after binary search title_candidates: 1159
after binary search body_candidates: 4669
anchor_candidates: 0
cos_candidates_to_add: 0
final score:0.5665862068965517
title_candidates: 9759
body_candidates: 813413
body_candidates_bi_gram: 112
title_candidates_bi_gram: 0
after binary search title_candidates: 603
after binary search body_candidates: 4995
anchor_candidates: 0
cos_candidates_to_add: 0
title_candidates: 42617
body_candidates: 570026
body_candidates_bi_gram: 1624
title_candidates_bi_gram: 10
after binary search title_candidates: 603
after binary search body_candidates: 1624
anchor_candidates: 0
cos_candidates_to_add: 0
title_candidates: 2387
body_candidates: 471533
body_candidates_bi_gram: 1400
title_candidates_bi_gram: 33
after binary search title_candidates: 603
after binary search body_candidates: 1400
anchor_candidates: 0
cos_candidates_to_add: 0
title_candidates: 7
body_candidates: 2393
body_candidates_bi_gram: 0
title_candidates_bi_gram: 0
after binary search title_candidat

[I 2023-01-15 07:29:58,604] Trial 149 finished with value: 0.5559655172413792 and parameters: {'max_docs_from_binary_title': 603, 'max_docs_from_binary_body': 4995, 'bm25_body_weight': 6.923091165583881, 'bm25_title_weight': 7.658368610051589, 'bm25_body_bi_weight': 2.9089277200565906, 'bm25_title_bi_weight': 8.71083472741107, 'body_cosine_score': 9.528132798540355, 'title_cosine_score': 5.836889203533235, 'pr_weight': 0.539103794876653, 'pv_weight': 2.7969759291886653}. Best is trial 146 with value: 0.5772068965517242.


after binary search title_candidates: 603
after binary search body_candidates: 4995
anchor_candidates: 0
cos_candidates_to_add: 0
final score:0.5559655172413792
Number of completed trials: 150
Best trial:
	Best Score: 0.5772068965517242
	Best Params: 
    max_docs_from_binary_title: 877
    max_docs_from_binary_body: 4797
    bm25_body_weight: 6.868827373169719
    bm25_title_weight: 8.522536016197188
    bm25_body_bi_weight: 3.263402828780264
    bm25_title_bi_weight: 8.057043505137194
    body_cosine_score: 9.739475232590543
    title_cosine_score: 4.373553673472577
    pr_weight: 1.2187494507831327
    pv_weight: 4.680150743637637
